# 图像识别问题介绍及经典数据集

## CIFAR数据集

1. CIFAR 数据集(CIFAR-10【60000张】, CIFAR-100)
2. 大小$32\times 32$
3. 人工标注的正确率94%

## ImageNet

1. 1500W张图片
2. ImageNet每年都会举办比赛
3. [ImageNet](http://image-net.org/challenges/LSVRC/)

# 卷积神经网络介绍

1. 全连接层网络

问题：参数太多

2. 卷积神经网络

特点：减少参数

## 结构

1. 输入层
2. 卷积层
3. 池化层
4. 全连接层
5. Softmax层

# 卷积神经网络常用结构

## 卷积层

**过滤器**

过滤器有三个参数，长、宽和深度

1. 长宽是过滤器在输入矩阵上的大小。
2. 深度是输出矩阵的深度。

$$
g(i)=f(\sum_{x=1}^{2}\sum_{y=1}^{2}\sum_{z=1}^{3}a_{x,y,z}\times w_{x,y,z}^i+b^i)
$$

**如何理解这个深度。深度就是一次卷积核产生的输出个数。**

Q:为了避免尺寸变化可以采用边界填0补充。

Q:移动步长也可以设置

**卷积层** 的参数是共享的。所以一个$2\times 2$的卷积核只有4个参数。

举个例子对于输入为$5\times 5 \times 3$的输入。如果输出的深度为$16$那么卷积核的参数数量为

$$
5\times 5\times 3\times 16 + 16 = 1216
$$

最后加的16是偏移量。

下面看看具体代码

In [ ]:
import tensorflow as tf

# 卷积核
filter_weight = tf.get_variable(
    "weights", [5,5,3,16], #5,5是长宽，3是当前层深度。16是输出层深度
    initializer=tf.truncated_normal_initializer(stddev=0.1))

biases = tf.get_variable(
    "biases", [16], initializer=tf.constant_initializer(0.1))


conv = tf.nn.conv2d(
    input, filter_weight, strides=[1,1,1,1], # 步长，4个维度上的步长。但是第一个和最后一个参数必须为1，因为步长只对长宽有效。
    padding="SAME") #SAME 是添加全0，VALID是不添加

bias = tf.nn.bias_add(conv, biases)
actived_conv = tf.nn.relu(bias) # relu is a activate function.

## 池化层

**作用：**

1. 非常有效的缩小矩阵的尺寸。
2. 加快计算。
3. 防止过拟合。

和卷积层不同的是，池化层的计算更为简单。

1. 最大池化层。(取最大值)
2. 平均池化层。(取平均值)

**池化层的过滤器与卷积层的过滤器不同的地方在于。池化层的过滤器之作用于当前层，不跨层。**

*池化层虽然可以减小深度，但是一般不这样做*

有人认为池化层对模型影响不大,参考SpringenbergJT,DosovitskiyA,BroxT,et al. Striving for Simplic。 The All Convolutional Net [J]. Eprint Arxiv, 2014.

**Dropout**

随机的将节点的输出改为0，避免过拟合。一般只在全连接层使用。

看看代码

In [ ]:
pool = tf.nn.max_pool(
    actived_conv, ksize=[1,3,3,1],
    strides=[1,2,2,1], padding="SAME")

# 经典卷积网络模型

Q: 神经网络的组合形式那么多。具体用什么模型呢？

## LeNet-5

1998年提出的模型，在MNIST上正确率为99.2%. 这个模型总共7层。

### 重新写inference

In [ ]:
import tensorflow as tf
# 设定神经网络的参数
INPUT_NODE = 784
OUTPUT_NODE = 10

IMAGE_SIZE = 28
NUM_CHANNELS = 1
NUM_LABELS = 10

CONV1_DEEP = 32
CONV1_SIZE = 5

CONV2_DEEP = 64
CONV2_SIZE = 5

FC_SIZE = 512

# 定义前向传播的过程

def inference(input_tensor, train, regularizer):
    with tf.variable_scope('layer1-conv1'):
        conv1_weights = tf.get_variable(
            "weight", [CONV1_SIZE, CONV1_SIZE, NUM_CHANNELS, CONV1_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases = tf.get_variable("bias", [CONV1_DEEP], initializer=tf.constant_initializer(0.0))
        conv1 = tf.nn.conv2d(input_tensor, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))

    with tf.name_scope("layer2-pool1"):
        pool1 = tf.nn.max_pool(relu1, ksize = [1,2,2,1],strides=[1,2,2,1],padding="SAME")

    with tf.variable_scope("layer3-conv2"):
        conv2_weights = tf.get_variable(
            "weight", [CONV2_SIZE, CONV2_SIZE, CONV1_DEEP, CONV2_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases = tf.get_variable("bias", [CONV2_DEEP], initializer=tf.constant_initializer(0.0))
        conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))

    with tf.name_scope("layer4-pool2"):
        pool2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        pool_shape = pool2.get_shape().as_list()
        nodes = pool_shape[1] * pool_shape[2] * pool_shape[3] # 把输出拉成一个一维向量。
        reshaped = tf.reshape(pool2, [pool_shape[0], nodes]) # pool_shape[0] 为一个batch中数据的个数.

    with tf.variable_scope('layer5-fc1'):
        fc1_weights = tf.get_variable("weight", [nodes, FC_SIZE],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc1_weights))
        fc1_biases = tf.get_variable("bias", [FC_SIZE], initializer=tf.constant_initializer(0.1))

        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
        if train: fc1 = tf.nn.dropout(fc1, 0.5) # 随机的将节点的输出改为0，避免过拟合。一般只在全连接层使用。

    with tf.variable_scope('layer6-fc2'):
        fc2_weights = tf.get_variable("weight", [FC_SIZE, NUM_LABELS],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc2_weights))
        fc2_biases = tf.get_variable("bias", [NUM_LABELS], initializer=tf.constant_initializer(0.1))
        logit = tf.matmul(fc1, fc2_weights) + fc2_biases

    return logit

### 重写train

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import LeNet5_infernece
import os
import numpy as np

# 定义神经网络相关的参数

BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.01
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 6000
MOVING_AVERAGE_DECAY = 0.99

# 定义训练过程

def train(mnist):
    # 定义输出为4维矩阵的placeholder
    x = tf.placeholder(tf.float32, [
            BATCH_SIZE,
            LeNet5_infernece.IMAGE_SIZE,
            LeNet5_infernece.IMAGE_SIZE,
            LeNet5_infernece.NUM_CHANNELS],
        name='x-input')
    y_ = tf.placeholder(tf.float32, [None, LeNet5_infernece.OUTPUT_NODE], name='y-input')

    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = LeNet5_infernece.inference(x,False,regularizer)
    global_step = tf.Variable(0, trainable=False)

    # 定义损失函数、学习率、滑动平均操作以及训练过程。
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
        staircase=True)

    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')

    # 初始化TensorFlow持久化类。
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)

            reshaped_xs = np.reshape(xs, (
                BATCH_SIZE,
                LeNet5_infernece.IMAGE_SIZE,
                LeNet5_infernece.IMAGE_SIZE,
                LeNet5_infernece.NUM_CHANNELS))
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: reshaped_xs, y_: ys})

            if i % 1000 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
# 程序住入口
def main(argv=None):
    mnist = input_data.read_data_sets("./data", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    main()

### 所以一般神经网络的模型满足下面的正则表达式

输入层->((卷积层+)->(池化层？))+->(全连接层+)

### 其他类似网络

1. AlexNet
2. ZF Net
3. VGGNet

## Inception-v3模型

与前面的模型不同之处是这里采用并联的方式。

![inception](incep.png)

1. 使用不同尺寸的过滤器。
2. 如果采用padding为0，移动步长为1，1。那么最后不同过滤器的产出维度是一样的。这样就可以在深度上把他们拼接起来。

### 使用Tensorflow-Slim工具来产生 inception卷积层(因为手动国语复杂。)

In [ ]:
import tensorflow as tf

with tf.variable_scope(scope_name):
    weights = tf.get_variable("weight", ...)
    biases = tf.get_variable("bias", ...)
    conv = tf.nn.conv2d(...)
    relu = tf.nn.relu(tf.nn.bias_add(conv, biases))

## 上面的代码等价于

slim = tf.contrib.slim
net = slim.conv2d(input, 32, [3,3])

### 来看看一段slim创建一部分模块的代码

完整的代码请参考
[github(slim)](https://github.com/tensorflow/models/blob/master/research/slim/nets/inception_v3.py)

In [ ]:
import tensorflow as tf

slim = tf.contrib.slim

with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d],
                    stride=1,padding='VALID'):
    net = 上一层的输出节点矩阵
    with tf.variable_scope("Mized_7c"):
        with tf.variable_scope("Branch_0"):
            branch_0 = slim.conv2d(net, 320, [1,1], scope='Conv2d_0a_1x1')

        with tf.variable_scope("Branch_1"):
            branch_1 = slim.conv2d(net, 384, [1,1], sclpe="Conv2d_0a_1x1")
            #拼接

            branch_1 = tf.concat(3, [slim.conv2d(branch_1, 384, [1, 3], scope='Conv2d_0b_1x3'),
                                     slim.conv2d(branch_1, 384, [3, 1], scope='Conv2d_0c_3x1')])

        with tf.variable_scope('Branch_2'):
            branch_2 = slim.conv2d(
                net, 448, [1, 1], scope='Conv2d_0a_1x1')
            branch_2 = slim.conv2d(
                branch_2, 384, [3, 3], scope='Conv2d_0b_3x3')
            branch_2 = tf.concat(3, [
                slim.conv2d(branch_2, 384, [1,3], scope='Conv2d_0c_1x3'),
                slim.conv2d(branch_2, 384, [3,1], scope='Conv2d_0d_3x1')])

        with tf.variable_scope('Branch_3'):
            branch_3 = slim.avg_pool2d(
                net, [3, 3], scope='AvgPool_0a_3x3')
            branch_3 = slim.conv2d(
                branch_3, 192, [1, 1], scope='Conv2d_0b_1x1')

        net = tf.concat(3, [branch_0, branch_1, branch_2, branch_3])

# 卷积神经网络迁移学习

提出原因：现在机器学习的模型逐年增大。有没有办法，将一个问题上训练好的模型通过简单的调整使用于一个新的问题。

1. 一般来说训练数据足够的话。迁移学习不如完全重新学习。
2. 迁移学习需要的时间和训练样本数小于完整训练。

## 下面的代码实现迁移学习

首先通过

wget http://download.tensorflow.org/example_images/flower_photos.tgz
tar zxf flower_photos.tgz

下载数据，然后将数据转换为需要的输入格式

In [ ]:
import glob
import os.path
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile

# 原始输入数据的目录，这个目录下有5个子目录，每个子目录底下保存这属于该
# 类别的所有图片。
INPUT_DATA = '../../datasets/flower_photos'
# 输出文件地址。我们将整理后的图片数据通过numpy的格式保存。
OUTPUT_FILE = '../../datasets/flower_processed_data.npy'

# 测试数据和验证数据比例。
VALIDATION_PERCENTAGE = 10
TEST_PERCENTAGE = 10

定义数据处理过程

# 读取数据并将数据分割成训练数据、验证数据和测试数据。
def create_image_lists(sess, testing_percentage, validation_percentage):
    sub_dirs = [x[0] for x in os.walk(INPUT_DATA)]
    is_root_dir = True

    # 初始化各个数据集。
    training_images = []
    training_labels = []
    testing_images = []
    testing_labels = []
    validation_images = []
    validation_labels = []
    current_label = 0

    # 读取所有的子目录。
    for sub_dir in sub_dirs:
        if is_root_dir:
            is_root_dir = False
            continue

        # 获取一个子目录中所有的图片文件。
        extensions = ['jpg', 'jpeg', 'JPG', 'JPEG']
        file_list = []
        dir_name = os.path.basename(sub_dir)
        for extension in extensions:
            file_glob = os.path.join(INPUT_DATA, dir_name, '*.' + extension)
            file_list.extend(glob.glob(file_glob))
        if not file_list: continue
        print "processing:", dir_name

        i = 0
        # 处理图片数据。
        for file_name in file_list:
            i += 1
            # 读取并解析图片，将图片转化为299*299以方便inception-v3模型来处理。
            image_raw_data = gfile.FastGFile(file_name, 'rb').read()
            image = tf.image.decode_jpeg(image_raw_data)
            if image.dtype != tf.float32:
                image = tf.image.convert_image_dtype(image, dtype=tf.float32)
            image = tf.image.resize_images(image, [299, 299])
            image_value = sess.run(image)

            # 随机划分数据聚。
            chance = np.random.randint(100)
            if chance < validation_percentage:
                validation_images.append(image_value)
                validation_labels.append(current_label)
            elif chance < (testing_percentage + validation_percentage):
                testing_images.append(image_value)
                testing_labels.append(current_label)
            else:
                training_images.append(image_value)
                training_labels.append(current_label)
            if i % 200 == 0:
                print i, "images processed."
        current_label += 1

    # 将训练数据随机打乱以获得更好的训练效果。
    state = np.random.get_state()
    np.random.shuffle(training_images)
    np.random.set_state(state)
    np.random.shuffle(training_labels)

    return np.asarray([training_images, training_labels,
                       validation_images, validation_labels,
                       testing_images, testing_labels])

#运行数据处理过程
with tf.Session() as sess:
    processed_data = create_image_lists(sess, TEST_PERCENTAGE, VALIDATION_PERCENTAGE)
    # 通过numpy格式保存处理后的数据。
    np.save(OUTPUT_FILE, processed_data)

运行代码，可以把输入数据划分为训练、验证、测试三个数据集。并将图片从jpg转化为inception-v3模型需要的$299\times 299 \times 3$的数字矩阵。

## 下载google训练好的Inception-v3模型

wget http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz

tar xzf inception_v3_2016_08_28.tar.gz

## 通过以下代码完成迁移学习

定义训练过程中将要使用到的常量。
因为GitHub无法保存大于100M的文件，所以在运行时需要先自行从Google下载inception_v3.ckpt文件。

In [ ]:
import glob
import os.path
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile
import tensorflow.contrib.slim as slim

# 加载通过TensorFlow-Slim定义好的inception_v3模型。
import tensorflow.contrib.slim.python.slim.nets.inception_v3 as inception_v3

# 处理好之后的数据文件。
INPUT_DATA = '../../datasets/flower_processed_data.npy'
# 保存训练好的模型的路径。
TRAIN_FILE = 'train_dir/model'
# 谷歌提供的训练好的模型文件地址。因为GitHub无法保存大于100M的文件，所以
# 在运行时需要先自行从Google下载inception_v3.ckpt文件。
CKPT_FILE = '../../datasets/inception_v3.ckpt'

# 定义训练中使用的参数。
LEARNING_RATE = 0.0001
STEPS = 300
BATCH = 32
N_CLASSES = 5

# 不需要从谷歌训练好的模型中加载的参数。
CHECKPOINT_EXCLUDE_SCOPES = 'InceptionV3/Logits,InceptionV3/AuxLogits'
# 需要训练的网络层参数明层，在fine-tuning的过程中就是最后的全联接层。
TRAINABLE_SCOPES='InceptionV3/Logits,InceptionV3/AuxLogit'

获取所有需要从谷歌训练好的模型中加载的参数。

In [ ]:
def get_tuned_variables():
    exclusions = [scope.strip() for scope in CHECKPOINT_EXCLUDE_SCOPES.split(',')]

    variables_to_restore = []
    # 枚举inception-v3模型中所有的参数，然后判断是否需要从加载列表中移除。
    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        if not excluded:
            variables_to_restore.append(var)
    return variables_to_restore

获取所有需要训练的变量列表。

In [ ]:
def get_trainable_variables():
    scopes = [scope.strip() for scope in TRAINABLE_SCOPES.split(',')]
    variables_to_train = []

    # 枚举所有需要训练的参数前缀，并通过这些前缀找到所有需要训练的参数。
    for scope in scopes:
        variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope)
        variables_to_train.extend(variables)
    return variables_to_train

定义训练过程。

In [ ]:
def main():
    # 加载预处理好的数据。
    processed_data = np.load(INPUT_DATA)
    training_images = processed_data[0]
    n_training_example = len(training_images)
    training_labels = processed_data[1]

    validation_images = processed_data[2]
    validation_labels = processed_data[3]

    testing_images = processed_data[4]
    testing_labels = processed_data[5]
    print("%d training examples, %d validation examples and %d testing examples." % (
        n_training_example, len(validation_labels), len(testing_labels)))

    # 定义inception-v3的输入，images为输入图片，labels为每一张图片对应的标签。
    images = tf.placeholder(tf.float32, [None, 299, 299, 3], name='input_images')
    labels = tf.placeholder(tf.int64, [None], name='labels')

    # 定义inception-v3模型。因为谷歌给出的只有模型参数取值，所以这里
    # 需要在这个代码中定义inception-v3的模型结构。虽然理论上需要区分训练和
    # 测试中使用到的模型，也就是说在测试时应该使用is_training=False，但是
    # 因为预先训练好的inception-v3模型中使用的batch normalization参数与
    # 新的数据会有出入，所以这里直接使用同一个模型来做测试。
    with slim.arg_scope(inception_v3.inception_v3_arg_scope()):
        logits, _ = inception_v3.inception_v3(
            images, num_classes=N_CLASSES, is_training=True)

    trainable_variables = get_trainable_variables()
    # 定义损失函数和训练过程。
    tf.losses.softmax_cross_entropy(
        tf.one_hot(labels, N_CLASSES), logits, weights=1.0)
    total_loss = tf.losses.get_total_loss()
    train_step = tf.train.RMSPropOptimizer(LEARNING_RATE).minimize(total_loss)

    # 计算正确率。
    with tf.name_scope('evaluation'):
        correct_prediction = tf.equal(tf.argmax(logits, 1), labels)
        evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # 定义加载Google训练好的Inception-v3模型的Saver。
    load_fn = slim.assign_from_checkpoint_fn(
      CKPT_FILE,
      get_tuned_variables(),
      ignore_missing_vars=True)

    # 定义保存新模型的Saver。
    saver = tf.train.Saver()

    with tf.Session() as sess:
        # 初始化没有加载进来的变量。
        init = tf.global_variables_initializer()
        sess.run(init)

        # 加载谷歌已经训练好的模型。
        print('Loading tuned variables from %s' % CKPT_FILE)
        load_fn(sess)

        start = 0
        end = BATCH
        for i in range(STEPS):
            _, loss = sess.run([train_step, total_loss], feed_dict={
                images: training_images[start:end],
                labels: training_labels[start:end]})

            if i % 30 == 0 or i + 1 == STEPS:
                saver.save(sess, TRAIN_FILE, global_step=i)

                validation_accuracy = sess.run(evaluation_step, feed_dict={
                    images: validation_images, labels: validation_labels})
                print('Step %d: Training loss is %.1f Validation accuracy = %.1f%%' % (
                    i, loss, validation_accuracy * 100.0))

            start = end
            if start == n_training_example:
                start = 0

            end = start + BATCH
            if end > n_training_example:
                end = n_training_example

        # 在最后的测试数据上测试正确率。
        test_accuracy = sess.run(evaluation_step, feed_dict={
            images: testing_images, labels: testing_labels})
        print('Final test accuracy = %.1f%%' % (test_accuracy * 100))

运行训练

In [ ]:
if __name__ == '__main__':
    main()